In [1]:
import os

In [2]:
%pwd

'c:\\Users\\chami\\Desktop\\Learning\\ML\\MLOps-and-CICD\\Chicken-Disease-Classification-Project\\research'

In [3]:
# change current working directory to project root directory 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\chami\\Desktop\\Learning\\ML\\MLOps-and-CICD\\Chicken-Disease-Classification-Project'

In [5]:
# update the entity
# entity folder contained the custom function return types
# the frozen means that  once an instance of the DataIngestionConfig class is created, its attributes cannot be modified.

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir = Path
    source_URL = str
    local_data_file = Path
    unzip_dir = Path

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
# update configuration manager
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # create atrifacts folder inside the root folder
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        # get all the data ingestion related paths in config.yaml file
        config = self.config.data_ingestion
        # create data_ingestion folder inside the artifacts folder
        create_directories([config.root_dir])

        # get all the data ingestion related paths to one variable class and return it
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )

        return data_ingestion_config


In [8]:
# execute the data ingestion

import os 
import urllib.request as request
import zipfile
from cnnClassifier import logger 
from cnnClassifier.utils.common import get_size

In [13]:
# update componenet folder
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) :
        self.config = config

    def download_file(self):
        # if the data set not downloaded previously
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"file already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        # create unzip folder artifacts/data_ingestion
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        # unzip the downloaded file and saved it in artifacts/data_ingestion folder
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [17]:
# create pipeline
try:
    config = ConfigurationManager() # create artifacts folder
    data_ingestion_config = config.get_data_ingestion_config() # return data ingestion related paths
    data_ingestion = DataIngestion(config=data_ingestion_config) # initialized data ingestion object with related paths 
    data_ingestion.download_file() # download the dataset 
    data_ingestion.extract_zip_file() # extract the downloaded data zip file to  artifacts/data_ingestion folder
except Exception as e:
    raise e

[2024-05-07 00:03:21,380: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-07 00:03:21,382: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-07 00:03:21,382: INFO: common: created directory at: artifacts]
[2024-05-07 00:03:21,382: INFO: common: created directory at: artifacts/data_ingestion]


[2024-05-07 00:03:24,830: INFO: 2611563425: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 15B6:7A708:34327:4222B:663920AB
Accept-Ranges: bytes
Date: Mon, 06 May 2024 18:33:24 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1261-QPG
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1715020404.921491,VS0,VE325
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a017ac30fb6bff61a240982730d09caa3a00a40b
Expires: Mon, 06 May 2024 18:38:24 GMT
Source-Age: 0

]
